<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#数据准备" data-toc-modified-id="数据准备-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>数据准备</a></span></li><li><span><a href="#创建表项" data-toc-modified-id="创建表项-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>创建表项</a></span></li><li><span><a href="#查看配置" data-toc-modified-id="查看配置-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>查看配置</a></span></li><li><span><a href="#搜索" data-toc-modified-id="搜索-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>搜索</a></span></li><li><span><a href="#近义词模型" data-toc-modified-id="近义词模型-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>近义词模型</a></span></li><li><span><a href="#结合近义词搜索" data-toc-modified-id="结合近义词搜索-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>结合近义词搜索</a></span><ul class="toc-item"><li><span><a href="#测试" data-toc-modified-id="测试-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>测试</a></span></li><li><span><a href="#实现" data-toc-modified-id="实现-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>实现</a></span></li><li><span><a href="#验证" data-toc-modified-id="验证-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>验证</a></span></li></ul></li></ul></div>

## Setup

In [5]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from datetime import datetime
import pprint
import gensim

In [6]:
es = Elasticsearch()

## 数据准备

In [7]:
!head /tmp/model_name.txt

CAT统计
获取上下行绑定的信道
RCC模板配置与配置检查
attenuation&equalizer
通话结束后准入带宽占用查询
*Node_cb068
配置BPI+认证白名单
防MAC欺骗验证:env_init(),execute(),check(),test_destory(),OWFunc
配置证书管理状态:OWFunc;env_init();execute();check();test_destory()
查询CM版本信息


## 创建表项

In [8]:
test_score_index = "modle_index"
type_name = "my_type"
field_name = "content"

In [9]:
my_data_settting = {
    "mappings": {
        type_name: {
            "properties": {
                field_name: {
                    "type": "text",
                    "analyzer": "ik_max_word",
                    "search_analyzer": "ik_smart"
                }
            }
        }
    }
}

In [100]:
es.indices.exists(index=test_score_index)

True

In [98]:
es.indices.delete(index=test_score_index)
es.indices.create(index=test_score_index, body=my_data_settting)

PUT http://localhost:9200/modle_index [status:400 request:0.005s]


RequestError: TransportError(400, 'resource_already_exists_exception', 'index [modle_index/Hm75P_vgSGGpAXmzW4tW8Q] already exists')

In [11]:
def decode_nginx_log(data_iter):
    for idx, row in enumerate(data_iter):
        # Filter out the below from each log line
        # id_label = row
        # cmd_format_t = str(row['cmd_format_t'])

        # Index for elasticsearch. Typically timestamp.
        # idx = ...

        es_fields_keys = (field_name,)
        es_fields_vals = (row,)

        # We return a dict holding values from each line
        es_nginx_d = dict(zip(es_fields_keys, es_fields_vals))

        # Return the row on each iteration
        yield idx, es_nginx_d   # <- Note the usage of 'yield'

def es_add_bulk():
    with open("/tmp/model_name.txt", "r") as f:
        data_iter = f.readlines()
        data_iter = [x.strip() for x in data_iter]

    # NOTE the (...) round brackets. This is for a generator.
    k = ({
            "_index": test_score_index,
            "_type" : type_name,
            "_id"   : idx,
            "_source": es_nginx_d,
         } for idx, es_nginx_d in decode_nginx_log(data_iter))

    helpers.bulk(es, k)

# Now, just run it.
es_add_bulk()

## 查看配置

In [12]:
pprint.pprint(es.indices.get_mapping(index=test_score_index))
pprint.pprint(es.indices.get_settings(index=test_score_index))

{'modle_index': {'mappings': {'my_type': {'properties': {'content': {'analyzer': 'ik_max_word',
                                                                     'search_analyzer': 'ik_smart',
                                                                     'type': 'text'}}}}}}
{'modle_index': {'settings': {'index': {'creation_date': '1525663749971',
                                        'number_of_replicas': '1',
                                        'number_of_shards': '5',
                                        'provided_name': 'modle_index',
                                        'uuid': 'Hm75P_vgSGGpAXmzW4tW8Q',
                                        'version': {'created': '6020399'}}}}}


In [13]:
es.search(index=test_score_index, body={"query": {"match_all": {}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '0',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': 'CAT统计'},
    '_type': 'my_type'},
   {'_id': '14',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '配置EQAM信道模板'},
    '_type': 'my_type'},
   {'_id': '19',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '配置上行信道绑定的频谱管理策略组'},
    '_type': 'my_type'},
   {'_id': '22',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '配置RX光机的光路AGC增益控制范围'},
    '_type': 'my_type'},
   {'_id': '24',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '清除准入控制统计'},
    '_type': 'my_type'},
   {'_id': '25',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': 'CM-REG-STATUS模板'},
    '_type': 'my_type'},
   {'_id': '26',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '配置负载均衡策略'},
    '_

## 搜索

In [14]:
es.search(index=test_score_index, explain = False,size=3, body={
    "query": {
        "match": {
            field_name: {
                "query": "配置 sfid"
            }
        }
    },
    "highlight": {
        "fields" : {
            field_name : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '37',
    '_index': 'modle_index',
    '_score': 4.8841333,
    '_source': {'content': '获取动态流sfid信息'},
    '_type': 'my_type',
    'highlight': {'content': ['获取动态流<em>sfid</em>信息']}},
   {'_id': '620',
    '_index': 'modle_index',
    '_score': 2.126432,
    '_source': {'content': '配置vlan'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>配置</em>vlan']}},
   {'_id': '2',
    '_index': 'modle_index',
    '_score': 2.1227982,
    '_source': {'content': 'RCC模板配置与配置检查'},
    '_type': 'my_type',
    'highlight': {'content': ['RCC模板<em>配置</em>与<em>配置</em>检查']}}],
  'max_score': 4.8841333,
  'total': 192},
 'timed_out': False,
 'took': 2}

## 近义词模型

In [15]:
bin_file = '/root/workspace/mount/code/gensim/med256_5.model.bin'
word_vectors = gensim.models.Word2Vec.load(bin_file)

In [16]:
"设置" in word_vectors.wv.vocab

True

In [17]:
word_vectors.wv.most_similar("设置", topn=10)

[('修改', 0.523277759552002),
 ('设定', 0.39470696449279785),
 ('配置', 0.37758272886276245),
 ('更改', 0.35820212960243225),
 ('指定', 0.3255549371242523),
 ('小时数', 0.30822598934173584),
 ('为', 0.30258968472480774),
 ('调整', 0.3020206093788147),
 ('取值', 0.2891148030757904),
 ('无穷大', 0.28566598892211914)]

In [18]:
[x[0] for x in word_vectors.wv.most_similar("设置", topn=10) if len(x[0]) > 1]

['修改', '设定', '配置', '更改', '指定', '小时数', '调整', '取值', '无穷大']

## 结合近义词搜索

### 测试

In [127]:
es.search(index=test_score_index, explain = False,size=10, body={
    "query": {
        "bool": {
            "should": [
                {"constant_score": {"filter": {"term": {field_name: "清除"}}, "boost": 1.2}},
                {"constant_score": {"filter": {"term": {field_name: "查询"}}, "boost": 1.3}}
            ]
        }
    },
    "highlight": {
        "fields" : {
            field_name : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '116',
    '_index': 'modle_index',
    '_score': 1.3,
    '_source': {'content': 'monitor查询与断言_EQ'},
    '_type': 'my_type',
    'highlight': {'content': ['monitor<em>查询</em>与断言_EQ']}},
   {'_id': '145',
    '_index': 'modle_index',
    '_score': 1.3,
    '_source': {'content': '查询VOD_input统计'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>查询</em>VOD_input统计']}},
   {'_id': '166',
    '_index': 'modle_index',
    '_score': 1.3,
    '_source': {'content': '首次查询判断EQAM信道统计'},
    '_type': 'my_type',
    'highlight': {'content': ['首次<em>查询</em>判断EQAM信道统计']}},
   {'_id': '206',
    '_index': 'modle_index',
    '_score': 1.3,
    '_source': {'content': 'monitor查询与断言'},
    '_type': 'my_type',
    'highlight': {'content': ['monitor<em>查询</em>与断言']}},
   {'_id': '254',
    '_index': 'modle_index',
    '_score': 1.3,
    '_source': {'content': '查询及统计gate信息--通话过程中'},
    '_type': 'my_t

In [154]:
es.search(index=test_score_index, explain = False,size=1, body={
    "query": {
        "match": {
            field_name: {
                "query": "设定  验证"
            }
        }
    },
    "highlight": {
        "pre_tags" : ["<b>"],
        "post_tags" : ["</b>"],
        "fields" : {
            field_name : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '718',
    '_index': 'modle_index',
    '_score': 5.3301387,
    '_source': {'content': 'NIT验证参数'},
    '_type': 'my_type',
    'highlight': {'content': ['NIT<b>验证</b>参数']}}],
  'max_score': 5.3301387,
  'total': 14},
 'timed_out': False,
 'took': 112}

In [21]:
' '.join([x[0] for x in word_vectors.wv.most_similar("验证", topn=10) if len(x[0]) > 1])
' '.join([x[0] for x in word_vectors.wv.most_similar("设定", topn=10) if len(x[0]) > 1])

'测试 上网 正常 定量 状态刷新 ipsubnet-vlan 检查 svrp 可靠性测试 正确处理'

'设置 aucmodifyfldlist 报文大小 接近 -tc 值未 _NUM_pps 文件大小 绝对 温度'

In [22]:
es.search(index=test_score_index, explain = False,size=1, body={
    "query": {
        "bool": {
            "should": [
                {"match": { field_name: {"query": "设定  验证", "boost": 2}}},
                {"match": { field_name: "设置 aucmodifyfldlist 报文大小 接近 -tc 值未 _NUM_pps 文件大小 绝对 温度 测试 上网 正常 定量 状态刷新 ipsubnet-vlan 检查 svrp 可靠性测试 正确处理"}},
            ]
        }
    },
    "highlight": {
        "fields" : {
            field_name : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '255',
    '_index': 'modle_index',
    '_score': 11.838544,
    '_source': {'content': '严格组设置及验证'},
    '_type': 'my_type',
    'highlight': {'content': ['严格组<em>设置</em>及<em>验证</em>']}}],
  'max_score': 11.838544,
  'total': 98},
 'timed_out': False,
 'took': 4}

### 实现

In [23]:
input_str_a = "设定验证"
input_str_b = "设置验证"

In [24]:
def analyze_with_ik(i_str):
    ik_analyze = es.indices.analyze(body={"text": i_str, "analyzer": "ik_smart"})
    return [x['token'] for x in ik_analyze['tokens']]

In [25]:
analyze_with_ik(input_str_a)
analyze_with_ik(input_str_b)

['设定', '验证']

['设置', '验证']

In [26]:
def get_similar_word(i_list):
    assert isinstance(i_list, list)
    ret_list = []
    for v in i_list:
        if v in word_vectors.wv.vocab:
            ret_list.extend([x[0] for x in word_vectors.wv.most_similar(v, topn=10) if len(x[0]) > 1])
        else:
            ret_list.append(v)
    return ret_list

In [133]:
def get_similar_word_v2(i_list):
    assert isinstance(i_list, list)
    ret_list = []
    for v in i_list:
        if v in word_vectors.wv.vocab:
            ret_list.extend([(x[0], 1 / (i + 1)) for i, x in enumerate(word_vectors.wv.most_similar(v, topn=10)) if len(x[0]) > 1])
        else:
            ret_list.append((v, 1))
    return ret_list

In [136]:
get_similar_word_v2(["查看"])

[('观察', 1.0),
 ('检查', 0.5),
 ('查询', 0.3333333333333333),
 ('察看', 0.25),
 ('清除', 0.2),
 ('看到', 0.16666666666666666),
 ('显示', 0.14285714285714285),
 ('以及', 0.125)]

In [28]:
def get_similar_search_wrod(i_str):
    analyze_list = analyze_with_ik(i_str)
    similar_list = get_similar_word(analyze_list)
    analyze_similar_list = analyze_with_ik(' '.join(similar_list))
    return set(similar_list) & set(analyze_similar_list)

In [163]:
def get_similar_search_wrod_v2(i_str):
    # 分词
    analyze_list = analyze_with_ik(i_str)
    # 近义词
    similar_list = get_similar_word_v2(analyze_list)
    # 对近义词分词
    tmp = [x[0] for x in similar_list]
    analyze_similar_list = analyze_with_ik(' '.join(tmp))
    # 原始分词和近义词分词的交集
    analyze_similar_list = set(tmp) & set(analyze_similar_list)
    # 仅返回交集的list
    similar_list = [x for x in similar_list if x[0] in analyze_similar_list]
    # 将返回值格式化用于搜索
    return [{"constant_score": {"filter": {"term": {field_name: key}}, "boost": score}} for key, score in similar_list]

In [162]:
get_similar_search_wrod_v2(input_str_a)

[{'constant_score': {'boost': 1.0, 'filter': {'term': {'content': '设置'}}}},
 {'constant_score': {'boost': 0.5,
   'filter': {'term': {'content': 'aucmodifyfldlist'}}}},
 {'constant_score': {'boost': 0.25, 'filter': {'term': {'content': '接近'}}}},
 {'constant_score': {'boost': 0.125, 'filter': {'term': {'content': '文件大小'}}}},
 {'constant_score': {'boost': 0.1111111111111111,
   'filter': {'term': {'content': '绝对'}}}},
 {'constant_score': {'boost': 0.1, 'filter': {'term': {'content': '温度'}}}},
 {'constant_score': {'boost': 1.0, 'filter': {'term': {'content': '测试'}}}},
 {'constant_score': {'boost': 0.5, 'filter': {'term': {'content': '上网'}}}},
 {'constant_score': {'boost': 0.3333333333333333,
   'filter': {'term': {'content': '正常'}}}},
 {'constant_score': {'boost': 0.25, 'filter': {'term': {'content': '定量'}}}},
 {'constant_score': {'boost': 0.16666666666666666,
   'filter': {'term': {'content': 'ipsubnet-vlan'}}}},
 {'constant_score': {'boost': 0.14285714285714285,
   'filter': {'term': {'

In [159]:
[{"constant_score": {"filter": {"term": {field_name: key}}, "boost": score}} for key, score in get_similar_search_wrod_v2(input_str_a)]

[{'constant_score': {'boost': 1.0, 'filter': {'term': {'content': '设置'}}}},
 {'constant_score': {'boost': 0.5,
   'filter': {'term': {'content': 'aucmodifyfldlist'}}}},
 {'constant_score': {'boost': 0.25, 'filter': {'term': {'content': '接近'}}}},
 {'constant_score': {'boost': 0.125, 'filter': {'term': {'content': '文件大小'}}}},
 {'constant_score': {'boost': 0.1111111111111111,
   'filter': {'term': {'content': '绝对'}}}},
 {'constant_score': {'boost': 0.1, 'filter': {'term': {'content': '温度'}}}},
 {'constant_score': {'boost': 1.0, 'filter': {'term': {'content': '测试'}}}},
 {'constant_score': {'boost': 0.5, 'filter': {'term': {'content': '上网'}}}},
 {'constant_score': {'boost': 0.3333333333333333,
   'filter': {'term': {'content': '正常'}}}},
 {'constant_score': {'boost': 0.25, 'filter': {'term': {'content': '定量'}}}},
 {'constant_score': {'boost': 0.16666666666666666,
   'filter': {'term': {'content': 'ipsubnet-vlan'}}}},
 {'constant_score': {'boost': 0.14285714285714285,
   'filter': {'term': {'

In [150]:
{"constant_score": {"filter": {"term": {field_name: "清除"}}, "boost": 1.2}},
        {"constant_score": {"filter": {"term": {field_name: "查询"}}, "boost": 1.3}}

['正确处理']

In [92]:
def search_api(i_str):   
    ret = es.search(index=test_score_index, explain=False, size=10, body={
        "query": {
            "bool": {
                "should": [
                    {"match": { field_name: {"query": i_str, "boost": 4}}},
                    {"match": { field_name: ' '.join(get_similar_search_wrod(i_str))}},
                ]
            }
        },
        "highlight": {
            "fields" : {
                field_name : {}
            }
        }
    })
    return ret

In [164]:
get_similar_search_wrod_v2("查看")

[{'constant_score': {'boost': 1.0, 'filter': {'term': {'content': '观察'}}}},
 {'constant_score': {'boost': 0.5, 'filter': {'term': {'content': '检查'}}}},
 {'constant_score': {'boost': 0.3333333333333333,
   'filter': {'term': {'content': '查询'}}}},
 {'constant_score': {'boost': 0.25, 'filter': {'term': {'content': '察看'}}}},
 {'constant_score': {'boost': 0.2, 'filter': {'term': {'content': '清除'}}}},
 {'constant_score': {'boost': 0.16666666666666666,
   'filter': {'term': {'content': '看到'}}}},
 {'constant_score': {'boost': 0.14285714285714285,
   'filter': {'term': {'content': '显示'}}}},
 {'constant_score': {'boost': 0.125, 'filter': {'term': {'content': '以及'}}}}]

In [165]:
def search_api_v2(i_str):   
    ret = es.search(index=test_score_index, explain=False, size=10, body={
        "query": {
            "bool": {
                "should": [
                    {"match": { field_name: {"query": i_str, "boost": 4}}},
                    get_similar_search_wrod_v2(i_str)
                ]
            }
        },
        "highlight": {
            "fields" : {
                field_name : {}
            }
        }
    })
    return ret

In [166]:
search_api_v2("查看")

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '581',
    '_index': 'modle_index',
    '_score': 1.0,
    '_source': {'content': '观察tunner的频点信道输出功率'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>观察</em>tunner的频点信道输出功率']}},
   {'_id': '347',
    '_index': 'modle_index',
    '_score': 0.8333334,
    '_source': {'content': '查询因SAV检查丢弃报文数:env_init();execute()'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>查询</em>因SAV<em>检查</em>丢弃报文数:env_init();execute()']}},
   {'_id': '29',
    '_index': 'modle_index',
    '_score': 0.5,
    '_source': {'content': '下行通用modem负载均衡状态检查'},
    '_type': 'my_type',
    'highlight': {'content': ['下行通用modem负载均衡状态<em>检查</em>']}},
   {'_id': '92',
    '_index': 'modle_index',
    '_score': 0.5,
    '_source': {'content': '检查DIAGLOG-DETAIL-TYPE模板上报'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>检查</em>DIAGLOG-DETAIL-TYPE模板上报']}},
   {'_id': '190',
    '_index': 'modle_inde

In [137]:
def search_api_demo(i_str):   
    ret = search_api(i_str)
    print('\n'.join([x['_source']['content']  for x in ret['hits']['hits']]))

In [167]:
def search_api_demo_v2(i_str):   
    ret = search_api_v2(i_str)
    print('\n'.join([x['_source']['content']  for x in ret['hits']['hits']]))

In [158]:
[x['highlight']["content"][0] for x in search_api("显示流量增长")['hits']['hits']]

['端口<em>流量</em><em>增长</em><em>查询</em>',
 '发包仪<em>查询</em>报文实时<em>流量</em>统计',
 '检查上行收包<em>流量</em>',
 '组播<em>流量</em>检查',
 '组播接收<em>流量</em>检查',
 '组播<em>流量</em>时延丢包验证',
 '上下行发送IP报文并实时检查<em>流量</em>',
 '检查发包仪的实时<em>流量</em>并判断是否正确',
 '<em>查询</em>下行信道<em>占用</em>率并作<em>记录</em>',
 '准入带宽<em>占用</em><em>查询</em>']

In [33]:
input_str_a
input_str_b

'设定验证'

'设置验证'

In [34]:
search_api(input_str_a)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '255',
    '_index': 'modle_index',
    '_score': 11.838544,
    '_source': {'content': '严格组设置及验证'},
    '_type': 'my_type',
    'highlight': {'content': ['严格组<em>设置</em>及<em>验证</em>']}},
   {'_id': '718',
    '_index': 'modle_index',
    '_score': 10.660277,
    '_source': {'content': 'NIT验证参数'},
    '_type': 'my_type',
    'highlight': {'content': ['NIT<em>验证</em>参数']}},
   {'_id': '624',
    '_index': 'modle_index',
    '_score': 9.403228,
    '_source': {'content': '验证信道模板生效'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>验证</em>信道模板生效']}},
   {'_id': '602',
    '_index': 'modle_index',
    '_score': 8.456335,
    '_source': {'content': 'PAT版本号更新验证'},
    '_type': 'my_type',
    'highlight': {'content': ['PAT版本号更新<em>验证</em>']}},
   {'_id': '313',
    '_index': 'modle_index',
    '_score': 8.128364,
    '_source': {'content': '组播流量时延丢包验证'},
    '_type': 'my_type',
    'hig

In [35]:
search_api(input_str_b)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '255',
    '_index': 'modle_index',
    '_score': 16.191895,
    '_source': {'content': '严格组设置及验证'},
    '_type': 'my_type',
    'highlight': {'content': ['严格组<em>设置</em>及<em>验证</em>']}},
   {'_id': '200',
    '_index': 'modle_index',
    '_score': 11.155458,
    '_source': {'content': '满配置参数设置'},
    '_type': 'my_type',
    'highlight': {'content': ['满<em>配置</em>参数<em>设置</em>']}},
   {'_id': '718',
    '_index': 'modle_index',
    '_score': 10.660277,
    '_source': {'content': 'NIT验证参数'},
    '_type': 'my_type',
    'highlight': {'content': ['NIT<em>验证</em>参数']}},
   {'_id': '307',
    '_index': 'modle_index',
    '_score': 10.296492,
    '_source': {'content': '参数设置'},
    '_type': 'my_type',
    'highlight': {'content': ['参数<em>设置</em>']}},
   {'_id': '624',
    '_index': 'modle_index',
    '_score': 9.403228,
    '_source': {'content': '验证信道模板生效'},
    '_type': 'my_type',
    'highligh

In [36]:
def origin_search(i_str):
    return es.search(index=test_score_index, explain=True, size=3, body={
        "query": {
            "match": {
                field_name: {
                    "query": i_str
                }
            }
        },
        "highlight": {
            "fields" : {
                field_name : {}
            }
        }
    })

### 验证

In [110]:
get_similar_search_wrod(["查看"])

{'以及', '察看', '显示', '查询', '检查', '清除', '看到', '观察'}

通过这一组示例发现一个缺陷，“设置” 被扩展后，“设置” 和 “配置” 会重复命中，而 “设置” 的词频略低于 “视频” 导致 “满配置参数设置” 排序靠前

In [38]:
search_api("端口流量增长查询")
search_api("显示流量增长")

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '121',
    '_index': 'modle_index',
    '_score': 32.841454,
    '_source': {'content': '端口流量增长查询'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>端口</em><em>流量</em><em>增长</em><em>查询</em>']}},
   {'_id': '242',
    '_index': 'modle_index',
    '_score': 12.690411,
    '_source': {'content': '发包仪查询报文实时流量统计'},
    '_type': 'my_type',
    'highlight': {'content': ['发包仪<em>查询</em>报文实时<em>流量</em>统计']}},
   {'_id': '146',
    '_index': 'modle_index',
    '_score': 12.079941,
    '_source': {'content': '添加端口到vlan'},
    '_type': 'my_type',
    'highlight': {'content': ['添加<em>端口</em>到<em>vlan</em>']}},
   {'_id': '766',
    '_index': 'modle_index',
    '_score': 11.551895,
    '_source': {'content': '端口删除绑定'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>端口</em>删除绑定']}},
   {'_id': '399',
    '_index': 'modle_index',
    '_score': 11.03058,
    '_source': {'content': '不输

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '121',
    '_index': 'modle_index',
    '_score': 22.065779,
    '_source': {'content': '端口流量增长查询'},
    '_type': 'my_type',
    'highlight': {'content': ['端口<em>流量</em><em>增长</em><em>查询</em>']}},
   {'_id': '242',
    '_index': 'modle_index',
    '_score': 10.775113,
    '_source': {'content': '发包仪查询报文实时流量统计'},
    '_type': 'my_type',
    'highlight': {'content': ['发包仪<em>查询</em>报文实时<em>流量</em>统计']}},
   {'_id': '68',
    '_index': 'modle_index',
    '_score': 9.768267,
    '_source': {'content': '检查上行收包流量'},
    '_type': 'my_type',
    'highlight': {'content': ['检查上行收包<em>流量</em>']}},
   {'_id': '179',
    '_index': 'modle_index',
    '_score': 9.376853,
    '_source': {'content': '组播流量检查'},
    '_type': 'my_type',
    'highlight': {'content': ['组播<em>流量</em>检查']}},
   {'_id': '435',
    '_index': 'modle_index',
    '_score': 8.272935,
    '_source': {'content': '查询下行信道占用率并作记录'},
    '_ty

In [39]:
search_api("修改用户优先级")
search_api("配置优先级")

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '110',
    '_index': 'modle_index',
    '_score': 22.629465,
    '_source': {'content': '修改用户优先级'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>修改</em><em>用户</em><em>优先级</em>']}},
   {'_id': '73',
    '_index': 'modle_index',
    '_score': 15.526941,
    '_source': {'content': '用户优先级'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>用户</em><em>优先级</em>']}},
   {'_id': '187',
    '_index': 'modle_index',
    '_score': 11.881593,
    '_source': {'content': '基于用户vlan与优先级映射'},
    '_type': 'my_type',
    'highlight': {'content': ['基于<em>用户</em>vlan与<em>优先级</em>映射']}},
   {'_id': '479',
    '_index': 'modle_index',
    '_score': 10.585472,
    '_source': {'content': '修改频谱模板'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>修改</em>频谱模板']}},
   {'_id': '557',
    '_index': 'modle_index',
    '_score': 10.248034,
    '_source': {'content': '配置网络侧用户'},
    '_ty

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '110',
    '_index': 'modle_index',
    '_score': 11.314733,
    '_source': {'content': '修改用户优先级'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>修改</em>用户<em>优先级</em>']}},
   {'_id': '659',
    '_index': 'modle_index',
    '_score': 10.8996935,
    '_source': {'content': '配置上行调度优先级映射'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>配置</em>上行调度<em>优先级</em>映射']}},
   {'_id': '532',
    '_index': 'modle_index',
    '_score': 10.139364,
    '_source': {'content': '配置高优先级呼叫统计周期'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>配置</em>高<em>优先级</em>呼叫统计周期']}},
   {'_id': '769',
    '_index': 'modle_index',
    '_score': 10.103866,
    '_source': {'content': '模板删除绑定'},
    '_type': 'my_type',
    'highlight': {'content': ['模板<em>删除</em><em>绑定</em>']}},
   {'_id': '773',
    '_index': 'modle_index',
    '_score': 10.007824,
    '_source': {'content': '删除未绑定域后再创

In [40]:
search_api("组播流量检查")
search_api("查看流量")

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '179',
    '_index': 'modle_index',
    '_score': 28.952242,
    '_source': {'content': '组播流量检查'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>组</em><em>播</em><em>流量</em><em>检查</em>']}},
   {'_id': '635',
    '_index': 'modle_index',
    '_score': 24.591267,
    '_source': {'content': '组播接收流量检查'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>组</em><em>播</em>接收<em>流量</em><em>检查</em>']}},
   {'_id': '198',
    '_index': 'modle_index',
    '_score': 22.39802,
    '_source': {'content': '组播节目在线检查'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>组</em><em>播</em>节目在线<em>检查</em>']}},
   {'_id': '420',
    '_index': 'modle_index',
    '_score': 16.909657,
    '_source': {'content': '组播全局配置'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>组</em><em>播</em>全局配置']}},
   {'_id': '68',
    '_index': 'modle_index',
    '_score': 15.931786,
    '_sourc

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '68',
    '_index': 'modle_index',
    '_score': 12.850026,
    '_source': {'content': '检查上行收包流量'},
    '_type': 'my_type',
    'highlight': {'content': ['<em>检查</em>上行收包<em>流量</em>']}},
   {'_id': '179',
    '_index': 'modle_index',
    '_score': 12.683599,
    '_source': {'content': '组播流量检查'},
    '_type': 'my_type',
    'highlight': {'content': ['组播<em>流量</em><em>检查</em>']}},
   {'_id': '121',
    '_index': 'modle_index',
    '_score': 11.53545,
    '_source': {'content': '端口流量增长查询'},
    '_type': 'my_type',
    'highlight': {'content': ['端口<em>流量</em>增长<em>查询</em>']}},
   {'_id': '242',
    '_index': 'modle_index',
    '_score': 10.775113,
    '_source': {'content': '发包仪查询报文实时流量统计'},
    '_type': 'my_type',
    'highlight': {'content': ['发包仪<em>查询</em>报文实时<em>流量</em>统计']}},
   {'_id': '635',
    '_index': 'modle_index',
    '_score': 10.250353,
    '_source': {'content': '组播接收流量检查'},
  

In [93]:
search_api_demo("查询VOD_output信息")

查询VOD_output统计
<em>查询</em><em>VOD_output</em>统计
清除VOD_output统计
<em>清除</em><em>VOD_output</em>统计
CM版本信息查询
CM版本<em>信息</em><em>查询</em>
CM远端信息查询
CM远端<em>信息</em><em>查询</em>
查询CM详细信息并判断Hqos信息是否正确
<em>查询</em>CM<em>详细信息</em>并判断Hqos<em>信息</em>是否正确
CM_PHY信息查询
CM_PHY<em>信息</em><em>查询</em>
查询EMTA信息
<em>查询</em>EMTA<em>信息</em>
查询信息输出
<em>查询</em><em>信息</em>输出
基于框槽信息查询
基于框槽<em>信息</em><em>查询</em>
CM详细信息查询（bonding&信道）
CM<em>详细信息</em><em>查询</em>（bonding&信道）


In [94]:
search_api_demo("查询EMTA统计")

查询EMTA信息
<em>查询</em><em>EMTA</em>信息
查询VOD_input统计
<em>查询</em>VOD_input<em>统计</em>
查询BC_input-ts统计
<em>查询</em>BC_input-ts<em>统计</em>
查询回传通道统计
<em>查询</em>回传通道<em>统计</em>
查询VOD_output统计
<em>查询</em>VOD_output<em>统计</em>
清除动态流统计
<em>清除</em>动态流<em>统计</em>
清除CM性能统计
<em>清除</em>CM性能<em>统计</em>
清除Gate统计
<em>清除</em>Gate<em>统计</em>
首次查询判断EQAM信道统计
首次<em>查询</em>判断EQAM信道<em>统计</em>
发包仪查询报文实时流量统计
发包仪<em>查询</em>报文实时流量<em>统计</em>


In [48]:
get_similar_search_wrod(["查看"])

{'以及', '察看', '显示', '查询', '检查', '清除', '看到', '观察'}

In [138]:
search_api_demo("获取端口流量增长")
print("--------------")
search_api_demo("端口流量增长信息查看")
print("--------------")
search_api_demo("显示流量增长")

端口流量增长查询
端口删除绑定
添加端口到vlan
发包仪查询报文实时流量统计
检查上行收包流量
cable端口统计
组播流量检查
不输入框槽端口，无法查询
配置端口角色
组播接收流量检查
--------------
端口流量增长查询
输入框槽端口，查询GCR信息
端口删除绑定
检查上行收包流量
组播流量检查
添加端口到vlan
发包仪查询报文实时流量统计
cable端口统计
组播接收流量检查
查询CM详细信息并判断Hqos信息是否正确
--------------
端口流量增长查询
发包仪查询报文实时流量统计
检查上行收包流量
组播流量检查
组播接收流量检查
组播流量时延丢包验证
上下行发送IP报文并实时检查流量
检查发包仪的实时流量并判断是否正确
查询下行信道占用率并作记录
准入带宽占用查询


In [168]:
search_api_demo_v2("获取端口流量增长")
print("--------------")
search_api_demo_v2("端口流量增长信息查看")
print("--------------")
search_api_demo_v2("显示流量增长")

端口流量增长查询
端口删除绑定
检查上行收包流量
cable端口统计
组播流量检查
发包仪查询报文实时流量统计
添加端口到vlan
配置端口角色
不输入框槽端口，无法查询
组播接收流量检查
--------------
端口流量增长查询
输入框槽端口，查询GCR信息
端口删除绑定
检查上行收包流量
组播流量检查
cable端口统计
发包仪查询报文实时流量统计
添加端口到vlan
配置端口角色
组播接收流量检查
--------------
端口流量增长查询
检查上行收包流量
组播流量检查
发包仪查询报文实时流量统计
组播接收流量检查
组播流量时延丢包验证
上下行发送IP报文并实时检查流量
检查发包仪的实时流量并判断是否正确
GQC生成正确性判断
配置文件生成


In [171]:
search_api_demo("查看")
print("------------")
search_api_demo_v2("查看")

清除Gate统计
gate统计清除
清除动态流统计
清除CM性能统计
清除准入控制统计
清除动态流DSX消息统计
清除回传通道统计
DCS检查
清除VOD_output统计
GC匹配检查
------------
观察tunner的频点信道输出功率
查询因SAV检查丢弃报文数:env_init();execute()
下行通用modem负载均衡状态检查
检查DIAGLOG-DETAIL-TYPE模板上报
检查发包仪的实时流量并判断是否正确
下行通用业务流负载均衡状态检查
检查组播参数配置
复位cm不检查
结果检查_vlan切换增强业务流信息校验_translate
检查SAMIX-TYPE1模板上报
